# ReAct ToolSelection approach with Azure AI Search
이 노트북에서는 ReAct를 활용해서 툴을 선택하는 기능을 사용해 볼 것이다. 사용자의 질문에 어떤 정보가 부족한지 확인하기 위해 여러차례 질문을 반복하며 평가하고 모든 정보가 수집되면 응답을 생성한다. ReAct로 툴의 **설명(description)**에만 의존하여 사용할 툴을 결정한다.

예제 코드에서는 두 가지 툴(Azure AI Search, CSV 룩업)을 사용해서 정보를 검색한다. 에이전트 처리는 LangChain의 **ZERO_SHOT_REACT_DESCRIPTION** 에이전트로 구현한다. 예제에는 무신 카페 검색용 CSV 파일을 검색하는 코드가 작성되어 있다.

# 사전 준비
이 파이썬 예제를 실행하려면 다음과 같은 환경이 필요하다:
- Azure AI Search 리소스의 엔드포인트 및 쿼리 API 키
- Azure OpenAI Service를 사용할 수 있는 [승인 완료](https://aka.ms/oai/access)된 Azure 구독
- Azure OpenAI Service에 배포된 `text-embedding-ada-002` Embeddings 모델. 이 모델의 API 버전은 `2023-05-15`을 사용했다. 배포 이름은 모델 이름과 동일하게 `text-embedding-ada-002`로 명명했다.
- Azure OpenAI Service 연동 및 모델 정보
  - OpenAI API 키
  - OpenAI Embeddings 모델의 배포 이름
  - OpenAI API 버전
- Python (이 예제는 버전 3.10.x로 테스트 했다.)

이 예제에서는 Visual Studio Code와 [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter)를 사용한다.

## 패키지 설치

In [ ]:
!pip install azure-search-documents==11.4.0
!pip install openai
!pip install langchain==0.0.350

In [ ]:
import azure.search.documents
print("azure.search.documents", azure.search.documents.__version__)
import langchain
print("langchain", langchain.__version__)

## 라이브러리 및 환경변수 불러오기

In [ ]:
import csv
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient
from azure.search.documents.models import (
    VectorizedQuery
)

from langchain.agents import (
    AgentType,
    Tool,
    initialize_agent,
)
from langchain.agents.mrkl import prompt
from langchain.chat_models import AzureChatOpenAI
#from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool

## Azure AI Search 연동 설정

In [ ]:
service_endpoint: str = "<Your search service endpoint>"
service_query_key: str = "<Your search service query key>"
index_name: str = "gptkbindex" # 자동 구축시 기본으로 설정

credential = AzureKeyCredential(service_query_key)

## Azure OpenAI 설정

In [ ]:
AZURE_OPENAI_API_KEY = "Your OpenAI API Key"
AZURE_OPENAI_ENDPOINT = "https://<Your OpenAI Service>.openai.azure.com/"
AZURE_OPENAI_CHATGPT_DEPLOYMENT = "chat" # 자동 구축시 기본으로 설정
AZURE_OPENAI_EMB_DEPLOYMENT="embedding" # 자동 구축시 기본으로 설정

In [ ]:
from openai import AzureOpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt  

openai_client = AzureOpenAI(
  api_key = AZURE_OPENAI_API_KEY,  
  api_version = "2024-02-01",
  azure_endpoint = AZURE_OPENAI_ENDPOINT
)

@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# title 필드와 content 필드의 Embeddings를 생성하는 함수
def generate_embeddings(text, model=AZURE_OPENAI_EMB_DEPLOYMENT):
    return openai_client.embeddings.create(input = [text], model=model).data[0].embedding

# 무신 검색 툴의 Retrieve 구현
무신 검색 툴은 Azure AI Search와 연동하여 검색 결과를 반환한다.

In [ ]:
def retrieve(query_text: str):
    search_client = SearchClient(service_endpoint, index_name, credential=credential)
    docs = search_client.search(
        search_text=query_text,
        filter=None,
        top=3,
        vector_queries=[VectorizedQuery(vector=generate_embeddings(query_text), k_nearest_neighbors=3, fields="embedding")]
    )
    results =[doc['sourcepage'] + ": " + nonewlines(doc['content']) for doc in docs]
    content = "\n".join(results)
    return content
    
def nonewlines(s: str) -> str:
    return s.replace('\n', ' ').replace('\r', ' ').replace('[', '【').replace(']', '】')

# 카페 검색 툴 정의
카페 검색 툴은 CSV 룩업을 사용해서 쿼리에 해당 카페 정보를 반환한다.

In [ ]:
class CafeSearchTool(BaseTool):
    data: dict[str, str] = {}
    name = "CafeSearchTool"
    description = "무신과 연관된 카페를 검색하는 데 특화된 도구입니다. 카페 검색 쿼리에는 무신의 **이름**만 입력해주세요."

    # Use the tool synchronously.
    def _run(self, query: str) -> str:
        filename = "data/restaurantinfo.csv"
        key_field = "name"
        try:
            with open(filename, newline='', encoding='utf-8') as csvfile:
                reader = csv.DictReader(csvfile)
                for row in reader:
                    self.data[row[key_field]] =  "\n".join([f"{i}:{row[i]}" for i in row])

        except Exception as e:
            print("File read error:", e)

        return self.data.get(query, "")

# Tool 정의
사용할 툴은 아래 2개로, 각각의 툴은 서로 다른 2종류의 방식으로 기술한다.
- 무신 검색 툴: Tool dataclass 방식을 사용해서 tools 안에 직접 기술
- 카페 검색 툴: BaseTool 클래스의 하위 클래스로 `CafeSearchTool`를 정의해서 기술

In [ ]:
# Tool dataclass 방식과 Subclassing the BaseTool class 방식의 차이점
tools = [
    Tool(name="PeopleSearchTool",
        func=retrieve,
        coroutine=retrieve,
        description="한국사 인물 정보 검색에 유용합니다. 사용자의 질문으로부터 검색 쿼리를 생성해서 검색을 수행합니다. 쿼리는 문자열만 받습니다."
        ),
    CafeSearchTool()
]

# LLM 정의

In [ ]:
llm = AzureChatOpenAI(
    azure_deployment=AZURE_OPENAI_CHATGPT_DEPLOYMENT,
    api_version="2024-02-01",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    openai_api_key=AZURE_OPENAI_API_KEY,
    #azure_ad_token_provider=self.openai_ad_token,
    temperature=0.0,
)

In [ ]:
q = "정중부, 이고와 함께 무신정변을 일으킨 인물의 이름과 이 인물의 출신지에 있는 카페의 이름을 알려줘."

# Agent 정의 및 실행

In [ ]:
SUFFIX = """
Answer should be in Korean.
"""
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    #agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    agent_kwargs=dict(suffix=SUFFIX + prompt.SUFFIX),
    handle_parsing_errors=True,
    max_iterations=5,
    early_stopping_method="generate",
)

result = agent_chain.run(q)
result